In [198]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Here we take height_weight_genders.csv as example.

In [338]:
# Load the data and convert it to metric system
def load_data(subsample=False, add_outlier = True):
    path = 'height_weight_genders.csv'
    data = np.genfromtxt(path, delimiter=',', skip_header=True, usecols=[1,2])
    height = data[:,0]
    weight = data[:,1]
    gender = np.genfromtxt(path, delimiter=',', skip_header=True, usecols=[0],
                          converters={0: lambda x : 1 if b'Male' in x else 0}) # col 0
    height *= 0.025
    weight *= 0.454
    if subsample:
        height = height[::20]
        weight = weight[::20]
    if add_outlier:
        height = np.concatenate([height, [1.1,1.2]])
        weight = np.concatenate([weight, [51.5/0.454, 55.3/0.454]])
    return height, weight, gender

def standerdize(x):
    x_mean = np.mean(x, axis=0)
    x_std = np.std(x, axis=0)
    x = (x - x_mean) / x_std
    return x, x_mean, x_std

In [348]:
height, weight, gender = load_data(subsample=False, add_outlier=False)
x, x_mean, x_std = standerdize(height)
y = weight
x_mat = np.c_[np.ones(len(x)),x]

In [201]:
DF = pd.DataFrame({'Height':height,'Weight':weight,'Gender':gender})
print(DF.head().to_string(index=False))

   Height      Weight  Gender
 1.846175  109.819678       1
 1.719548   73.688955       1
 1.852753   96.584348       1
 1.793274   99.899282       1
 1.747045   93.682809       1


In [202]:
print(x_mat)

[[ 1.          1.94406149]
 [ 1.          0.62753668]
 [ 1.          2.01244346]
 ...
 [ 1.         -0.64968792]
 [ 1.          0.69312469]
 [ 1.         -1.14970831]]


# Cost Function

## Mean Square Error

In [203]:
def mse(e):
    return np.mean(e**2)

## Mean Absolute Error

In [204]:
def mae(e):
    return np.mean(np.abs(e))

## Compute Loss

In [205]:
def loss(y, x_mat, w):
    e = y - x_mat.dot(w)
    return mse(e)

# Grid Search

In [296]:
def generate_w(num_intervals):
    w0 = np.linspace(-100,250,num_intervals)
    w1 = np.linspace(-150,120,num_intervals)
    return w0, w1

def grid_search(y, x_mat, w0, w1): #find the lowest loss
    losses = np.zeros((len(w0), len(w1)))
    for i, row in enumerate(w0):
        for j, col in enumerate(w1):
            w = np.array([row, col])
            losses[i, j] = loss(y, x_mat, w)
    return losses

def best_param(w0, w1, losses):
    min_row, min_col = np.unravel_index(np.argmin(losses), losses.shape)
    return losses[min_row, min_col], w0[min_row], w1[min_col]

def prediction(w0, w1, x_mean, x_std):
    x = np.arange(1.2, 2, 0.01)
    x_normalised = (x - x_mean) / x_std
    return x, w0 + w1 * x_normalised

In [297]:
def base_visualisation(grid_losses, w0_list, w1_list, x_mean, x_std, height, weight):
    w0, w1 = np.meshgrid(w0_list, w1_list)
    loss_star, w0_star, w1_star = best_param(w0_list, w1_list, grid_losses)
    
    fig = plt.figure()
    ax1 = fig.add_subplot(1,2,1)
    cp = ax1.contourf(w0, w1, grid_losses.T, cmap = plt.cm.jet)
    fig.colorbar(cp, ax=ax1)
    ax1.set_xlabel(r'$w_0$')
    ax1.set_ylabel(r'$w_1$')
    ax1.plot(w0_star, w1_star, marker = '*', color = 'r', markersize = 20)
    
    ax2 = fig.add_subplot(1,2,2)
    ax2.scatter(height, weight, marker='.', color = 'b', s = 5)
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    ax2.grid()
    
    return fig

In [461]:
def gradient_descent_visualisation(gradient_losses, gradient_w,
                                   grid_losses, grid_w0, grid_w1,
                                   x_mean, x_std, height, weight, n_iter=None):
    fig = base_visualisation(grid_losses, grid_w0, grid_w1, x_mean, x_std, height, weight)
    w_plotted = np.stack(gradient_w)
    if n_iter is not None:
        w_plotted = w_plotted[:n_iter]
    
    ax1, ax2 = fig.get_axes()[0], fig.get_axes()[2]
    ax1.plot(w_plotted[:,0], w_plotted[:,1], marker='o', color='w', markersize=5)
    pred_x, pred_y = prediction(w_plotted[-1,0], w_plotted[-1,1], x_mean, x_std)
    ax2.plot(pred_x, pred_y, 'r')
    
    return fig

# Gradient Descent
## Gradient
Suppose $y \approx f(x) = w_0+w_1x = \mathbf{X}w$  
$ \mathbf{X} \in R^{n \times (m+1)}, w \in R^{m \times 1}$  
$error = y-\mathbf{X}w$  
Here we take MSE as examples:  
$L = \frac{\sum (y-\mathbf{X}w)^2}{n}$  
then the gradient of the cost is:  
$\Delta L = \frac{\partial L}{\partial w} = -\frac{2}{n}\mathbf{X}^Te$

## Gradient Descent
$w^{k+1} = w^k - \tau \Delta L(w^k)$

In [329]:
def grad(y, x_mat, w):
    err = y - x_mat.dot(w)
    grad = -2 * x_mat.T.dot(err) / len(err)
    return grad, err

def gradient_descent(y, x_mat, tau, w_init, max_iter):
    w_ = [w_init]
    losses = []
    w = w_init
    for i in range(max_iter):
        gradient, err = grad(y, x_mat, w)
        loss = mse(err)
        w = w - tau * gradient
        w_.append(w)
        losses.append(loss)
        if (i+1)%5 == 0:
            print('Gradient Descent Iteration ({a}/{b}) : loss = {l}, w0 = {w0}, w1 = {w1}'.format(
                a = i+1, b = max_iter, l = loss.round(7), w0=w[0].round(7), w1=w[1].round(7)))
    return losses, w_

In [276]:
w0_grid, w1_grid = generate_w(num_intervals=250)
grid_losses = grid_search(y, x_mat, w0_grid, w1_grid)
lowest_loss, w0_best, w1_best = best_param(w0_grid, w1_grid, grid_losses)
print('Grid Search: loss*={l}, w0*={w0}, w1*={w1}'.format(l=lowest_loss, w0=w0_best, w1=w1_best))

Grid Search: loss*=30.998806863732035, w0*=72.89156626506022, w1*=13.734939759036138


In [349]:
w_init = np.array([0, 0])
gradient_losses, gradient_w = gradient_descent(y, x_mat, 0.05, w_init, 60)

Gradient Descent Iteration (5/60) : loss = 2421.4582301, w0 = 30.014594, w1 = 5.520077
Gradient Descent Iteration (10/60) : loss = 864.3525994, w0 = 47.7379116, w1 = 8.7796273
Gradient Descent Iteration (15/60) : loss = 321.423437, w0 = 58.2033534, w1 = 10.7043592
Gradient Descent Iteration (20/60) : loss = 132.1157436, w0 = 64.3830922, w1 = 11.8408941
Gradient Descent Iteration (25/60) : loss = 66.1082324, w0 = 68.0321661, w1 = 12.5120066
Gradient Descent Iteration (30/60) : loss = 43.0928363, w0 = 70.1869078, w1 = 12.9082918
Gradient Descent Iteration (35/60) : loss = 35.0678639, w0 = 71.4592612, w1 = 13.1422943
Gradient Descent Iteration (40/60) : loss = 32.2697291, w0 = 72.2105731, w1 = 13.2804704
Gradient Descent Iteration (45/60) : loss = 31.2940798, w0 = 72.6542153, w1 = 13.362062
Gradient Descent Iteration (50/60) : loss = 30.9538919, w0 = 72.9161816, w1 = 13.410241
Gradient Descent Iteration (55/60) : loss = 30.8352757, w0 = 73.0708701, w1 = 13.4386903
Gradient Descent Iterati

In [335]:
# Time Visualization
from ipywidgets import IntSlider, interact
def plot_figure(n_iter):
    fig = gradient_descent_visualisation(
        gradient_losses, gradient_w, grid_losses, grid_w0, grid_w1, x_mean, x_std, height, weight, n_iter)
    fig.set_size_inches(10.0, 6.0)

interact(plot_figure, n_iter=IntSlider(min=1, max=len(gradient_w), step=20))

interactive(children=(IntSlider(value=1, description='n_iter', max=61, min=1, step=20), Output()), _dom_classe…

<function __main__.plot_figure(n_iter)>

# Stochastic Gradient Descent
ref : https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/  
ref : https://towardsdatascience.com/batch-mini-batch-stochastic-gradient-descent-7a62ecba642a


In [416]:
def batches_generator(y, x_mat, batch_size, num_batches, shuffle=True):
    n = len(y)
    if shuffle:
        shuffle_indices = np.random.permutation(n)
        shuffled_y = y[shuffle_indices]
        shuffled_xmat = x_mat[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_xmat = x_mat
    
    for num_batch in range(num_batches):
        start_index = num_batch * batch_size
        end_index = min((num_batch + 1) * batch_size, n)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_xmat[start_index:end_index]

def grad_stoch(y, x_mat, w):
    err = y - x_mat.dot(w)
    grad = -2*x_mat.T.dot(err) / len(err)
    return grad, err

In [450]:
def Stochastic_Gradient_Descent(y, x_mat, w_init, batch_size, num_batches, max_iters, tau):
    w = w_init
    losses = []
    w_ = [w_init]
    for iter in range(max_iters):
        for y_batch, x_batch in batches_generator(y, x_mat, batch_size=batch_size, num_batches=num_batches):
            gradient, err = grad_stoch(y_batch, x_batch, w)
            w = w - tau * gradient
            error = y - x_mat.dot(w)
            loss = mse(error)
            w_.append(w)
            losses.append(loss)
        if (iter+1)%10 == 0:
            print('SGD({a}/{b}) : loss = {l}, w = {w}'.format(
                    a = iter+1, b = max_iters, l = loss, w = w))
    return losses, w_
            

In [480]:
sgd_losses, sgd_w = Stochastic_Gradient_Descent(y, x_mat, w_init, 1, 1, 50, 0.2)

SGD(10/50) : loss = 31.743098414021357, w = [72.50153946 14.06576071]
SGD(20/50) : loss = 41.364219721618106, w = [70.64961007 11.58233045]
SGD(30/50) : loss = 46.34647929591696, w = [72.57143602  9.59992925]
SGD(40/50) : loss = 62.891510866353485, w = [67.72152716 12.44619859]
SGD(50/50) : loss = 76.02942963316622, w = [76.68616692 19.28921571]


In [481]:
# Time Visualization
from ipywidgets import IntSlider, interact
def plot_figure(n_iter):
    fig = gradient_descent_visualisation(
        sgd_losses, sgd_w, grid_losses, grid_w0, grid_w1, x_mean, x_std, height, weight, n_iter)
    fig.set_size_inches(10.0, 6.0)

interact(plot_figure, n_iter=IntSlider(min=1, max=len(sgd_w), step=25))

interactive(children=(IntSlider(value=1, description='n_iter', max=51, min=1, step=25), Output()), _dom_classe…

<function __main__.plot_figure(n_iter)>

In [468]:
def MiniBatch_Gradient_Descent(y, x_mat, w_init, batch_size, num_batches, max_iters, tau):
    w = w_init
    losses = []
    w_ = [w_init]
    for iter in range(max_iters):
        for y_batch, x_batch in batches_generator(y, x_mat, batch_size=batch_size, num_batches=num_batches):
            gradient, err = grad_stoch(y_batch, x_batch, w)
            w = w - tau * gradient
            error = y - x_mat.dot(w)
            loss = mse(error)
            w_.append(w)
            losses.append(loss)
        if (iter+1)%10 == 0:
            print('Mini-Batch Gradient Descent({a}/{b}) : loss = {l}, w = {w}'.format(
                    a = iter+1, b = max_iters, l = loss, w = w))
    return losses, w_

In [484]:
mbgd_losses, mbgd_w = MiniBatch_Gradient_Descent(y, x_mat, w_init, 10, 1, 50, 0.05)

Mini-Batch Gradient Descent(10/50) : loss = 659.8360274565425, w = [48.4104024  10.33730829]
Mini-Batch Gradient Descent(20/50) : loss = 104.4053543590906, w = [65.00421908 11.2628669 ]
Mini-Batch Gradient Descent(30/50) : loss = 43.15885382047828, w = [70.05411178 12.10468233]
Mini-Batch Gradient Descent(40/50) : loss = 32.38258800736659, w = [72.15278517 12.92417756]
Mini-Batch Gradient Descent(50/50) : loss = 31.29569462361704, w = [72.5976721  13.67758857]


In [485]:
# Time Visualization
from ipywidgets import IntSlider, interact
def plot_figure(n_iter):
    fig = gradient_descent_visualisation(
        mbgd_losses, mbgd_w, grid_losses, grid_w0, grid_w1, x_mean, x_std, height, weight, n_iter)
    fig.set_size_inches(10.0, 6.0)

interact(plot_figure, n_iter=IntSlider(min=1, max=len(mbgd_w), step=25))

interactive(children=(IntSlider(value=1, description='n_iter', max=51, min=1, step=25), Output()), _dom_classe…

<function __main__.plot_figure(n_iter)>